# Learning LangChain

1. [Setting up OpenAI](#setting-up-openai)
2. [LangChain Chat API](#langchain-chat-api)
    - [Set up Model](#initializing-openai-chat-model-through-langchain)
    - [Prompt Templates](#prompt-templates)
3. [Output Parsing](#output-parsing)
    - [LLM response type](#llm-response-type)
    - [Response Schema](#response-schema)
    - [Parsing Response](#parsing-response)
4. [Memory](#memory)
    - [Saving Context](#saving-context)

# TO-DOs
1. implement token counts
    - look up LLM-specific tokenizers.
2. switch this notebook to free language models
3. look at course description/course syllabi
    - data retrieval? TIME CONSUMING TASK!
    - assist.org
        - manually choose a few courses to compare
        - purpose: to learn LLM tools
            - use in-context learning to ask a LLM whether it matches a course
            - if you had to give a similarity score, what would that be?
            - extract data?
            - both descriptions and syllabi (include courses that do NOT match)
            - what models perform well at this task? what ones do not?
4. further thoughts (for future features?)
    - email generation in advisor's writing style
    - play with in-context learning to duplicate writing styles

## Setting up OpenAI

In [1]:
import os
import openai
from google.generativeai import list_models

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
list(list_models())

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='Chat Bison',
       description='Chat-optimized generative language model.',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='Text Bison',
       description='Model targeted for text generation.',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtain a distributed representation of a text.',
       input_token_l

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## LangChain Chat API

### Initializing OpenAI Chat Model through LangChain

In [5]:
from langchain.chat_models import ChatOpenAI, ChatGooglePalm

In [6]:
# chat = ChatOpenAI(temperature=0, model=llm_model)
chat = ChatGooglePalm(temperature=0, model_name="models/chat-bison-001")
chat

ChatGooglePalm(client=<module 'google.generativeai' from '/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.12/site-packages/google/generativeai/__init__.py'>, temperature=0.0)

### Prompt Templates

In [73]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

In [8]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [9]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))])

In [10]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [11]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [12]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [13]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [14]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [15]:
customer_messages[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")

In [16]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"}]}


In [18]:
customer_response

ChatMessage(content='Sure, I can help you with that. Here is the text translated into American English in a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn\'t cover the cost of cleaning up my kitchen. I need your help right now, please!"\n\nI hope this is helpful!', role='1')

In [19]:
print(customer_response.content)

Sure, I can help you with that. Here is the text translated into American English in a calm and respectful tone:

"I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, please!"

I hope this is helpful!


In [20]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [21]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [22]:
service_response = chat(service_messages)
print(service_response.content)

{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"}]}
Ahoy there, matey! The warranty does not cover cleaning expenses for yer kitchen because it be yer fault that ye misused yer blender by forgettin' to put the lid on before startin' the blender. Tough luck! See ye later!


## Output Parsing
To deal with responses given by an LLM, we must give it instructions on how to
format the information that we wish to receive so that the response can be
parsed by LangChain.  Below is an overview of how that is done.

### LLM response type
Output from LLMs come as strings, so output parsers will be necessary to parse
the incoming data into a python dictionary

In [23]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [24]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [25]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think

#### Looking at the response more carefully

In [26]:
type(response.content)

str

In [27]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [28]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

### Response Schema
Response schema, in conjunction with the StructuredOutputParser, will allow us
to generate the format instructions that we can feed into prompts so that the
response can be parsed by LangChain.

In [29]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? \
                             Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product\
                                      to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or \
                                    price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [30]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [31]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?                              Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product                                      to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or                                     price, and output them as a comma separated Python list.
}
```


In [32]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [33]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [34]:
response = chat(messages)
print(response.content)

{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': 'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code sni

### Parsing Response

In [35]:
output_dict = output_parser.parse(response.content)

In [36]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ['slightly more expensive than the other leaf blowers out there']}

In [37]:
type(output_dict)

dict

In [38]:
output_dict.get('price_value') # notice that this is a list

['slightly more expensive than the other leaf blowers out there']

## Memory

LLMs are stateless, so each transaction is independent.  The way they appear to
have memory is because with each transaction, the entire conversation is
provided as context.

### !!!! NEED TO FIND TOKEN COUNTS

### Saving Context

Using LangChain's memory, we can store the entire conversation to be fed into
the LLM so that for each transaction, the LLM will know what has been said
before.

*Warning:* As the conversation gets longer and longer, the number of tokens
being sent to the LLM increases, and since language models price by token count,
the cost of providing the entire conversation as context can become
prohibitively expensive.  LangChain has some ways to mitigate this.

In [ ]:
del chat

In [63]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [205]:
style = "polite American English"

# prompt_template = """\
# You are a friendly conversational chatbot who is talkative \
# and provides lots of specific details from the current conversation history \
# below. Your talking style is """ + style + """. If you do not know the answer to a \
# question, truthfully say that you do not know.  Respond to the last line that the \
# human mentions.\nConversation History:\n\
# {history}\n\nLast line:\nHuman: {input}\nYou:"""

prompt_templates = ["""\
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its context. \
If the AI does not know the answer to a question or does not know how to \
respond, it truthfully says it does not know.  Complete the AI response to \
the last line that the human mentions.  Do not continue the human's portion \
of the conversation.\n\
Conversation History:\n{history}\n\nLast line:\nHuman: {input}"""
]

# This one is the best so far
prompt_templates.append("""\
You are a friendly conversational chatbot who is talkative \
and provides lots of specific details from the current conversation history \
below. Your talking style is """ + style + """. If you do not know the answer to a \
question, truthfully say that you do not know.  Respond to the last thing that the \
human says.\nConversation History:\n\
{history}\n\nLast line:\nHuman: {input}\nYou:""")

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=prompt_templates[-1]
)

In [211]:
# llm = ChatOpenAI(temperature=0, model=llm_model)
llm = ChatGooglePalm(temperature=0.5, model_name="models/chat-bison-001")
memory = ConversationBufferMemory(human_prefix="Human", ai_prefix="You")
conversation = ConversationChain(
    llm = llm,
    prompt=prompt,
    memory = memory,
    verbose = True      # verbose = True will show the entire conversation at each predict
)

In [212]:
conversation.predict(input="Hi, my name is Mark.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the human says.
Conversation History:


Last line:
Human: Hi, my name is Mark.
You:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': 'You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the human says.\nConversation History:\n\n\nLast line:\nHuman: Hi, my name is Mark.\nYou:'}]}

> Finished chain.


"Hi Mark, it's nice to meet you! How can I help you today?"

In [215]:
memory.buffer

"Human: Hi, my name is Mark.\nYou: Hi Mark, it's nice to meet you! How can I help you today?\nHuman: What is 1+1?\nYou: Hi Mark, it's nice to meet you too!\r\n\r\n1+1 is 2.\nHuman: What is my name?\nYou: Hi Mark, it's nice to meet you too!\r\n\r\n1+1 is 2.\r\n\r\nYou asked me what your name is. Your name is Mark."

In [213]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the human says.
Conversation History:
Human: Hi, my name is Mark.
You: Hi Mark, it's nice to meet you! How can I help you today?

Last line:
Human: What is 1+1?
You:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the human says.\nConversation History:\nHuman: Hi, my name is Mark.\nYou: Hi Mark,

"Hi Mark, it's nice to meet you too!\r\n\r\n1+1 is 2."

In [214]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the human says.
Conversation History:
Human: Hi, my name is Mark.
You: Hi Mark, it's nice to meet you! How can I help you today?
Human: What is 1+1?
You: Hi Mark, it's nice to meet you too!

1+1 is 2.

Last line:
Human: What is my name?
You:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "You are a friendly conversational chatbot who is talkative and provides lots of specific details from the current conversation history below. Your talking style is polite American English. If you do not know the answer to a question, truthfully say that you do not know.  Respond to the last thing that the hu

"Hi Mark, it's nice to meet you too!\r\n\r\n1+1 is 2.\r\n\r\nYou asked me what your name is. Your name is Mark."

In [194]:
print(memory.buffer)

Human: What is 1+1?
You: 1+1 is 2.
Human: What is my name?
You: Hi there! It's nice to meet you. I'm not sure what your name is, but I'm happy to chat with you regardless. What can I help you with today?


In [48]:
memory.load_memory_variables({})

# Why the {}? There are more advanced features with more sophisticated input

{'history': 'Human: What is 1+1?\nAI: 1+1 is 2.\nHuman: What is my name?\nAI: Human: What is my name?\nAI: I am sorry, I do not know your name.'}

In [49]:
memory = ConversationBufferMemory()

In [50]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [51]:
print(memory.buffer)

Human: Hi
AI: What's up


In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

### Conversation Buffer Window Memory

In [52]:
from langchain.memory import ConversationBufferWindowMemory

In [53]:
memory = ConversationBufferWindowMemory(k=1)

In [54]:
memory.save_context({"input": "Hi"}, {"output": "What's up?"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [55]:
memory.load_memory_variables({}) # notice that k is an request/response pair

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [56]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [62]:
conversation.predict(input="Hi, my name is Andrew. Can you greet me please?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew.  What is my name?
AI: Hi Andrew, it's nice to meet you. Your name is Andrew.
Human: Hi, my name is Andrew. Can you greet me please?
AI:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi, my name is Andrew.  What is my name?\nAI: Hi Andrew, it's nice to meet you. Your name is Andrew.\nHuman: Hi, my name is Andrew. Can you greet me please?\nAI:"}]}

> F

"Hi Andrew, it's nice to meet you. How can I help you today?"

In [58]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Not much, just hanging
AI: Cool
Human: What is 1+1?
AI:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': 'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is 1+1?\nAI:'}]}

> Finished chain.


'1+1=2.'

In [59]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1=2.
Human: What is my name?
AI:
{'context': '', 'examples': [], 'messages': [{'author': 'human', 'content': 'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: What is 1+1?\nAI: 1+1=2.\nHuman: What is my name?\nAI:'}]}

> Finished chain.


'Human: What is my name?\nAI: I am sorry, I do not know your name.'

### Conversation Token Buffer Memory

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=30
)

In [ ]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

### Conversation Summary Buffer

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

# Tokenizers

tiktoken is a fast open-source tokenizer by OpenAI.  This can be used to
tokenize and count tokens while accessing OpenAI's LLMs so that we do not exceed
the maximum token count (in addition to attempting to optimize token counts to
reduce costs).  The following are the encodings to be used for each of the
models:

| Encoding Name | OpenAI Model compatibility |
| --- | --- |
| `cl100k_base` | `gpt-4` `gpt-3.5-turbo` `text-embedding-ada-002` |
| `p50k_base` | Codex models, `text-davinci-002`, `text-davinci-003` |
| `r50k_base` or `gpt2` |	GPT-3 models (e.g. `davinci`) |

## tiktoken use

In [ ]:
import tiktoken

